In [1]:
%load_ext line_profiler
%pylab inline
from pycallgraph import PyCallGraph
from pycallgraph.output import GraphvizOutput

from __future__ import print_function
import cProfile

from JSAnimation import IPython_display
from matplotlib import animation
import time
import math

from species import *
from channels import *
from compartment import *
from membrane import *
from params import *

from csdmodel1d import *

import numpy as np
np.seterr(over ='warn',under='ignore')
import scipy as sp
from scipy.integrate import ode

#from dolfin import *  # FEM library
from ProgressBar import *

import logging
logging.getLogger("FFC").setLevel(logging.ERROR)

Populating the interactive namespace from numpy and matplotlib


In [2]:
model = CSDModelInterval(N=1000,dx=10e-6) # define the model

# Define the compartments, and the membranes
ecs = Compartment("ecs")
neuron = CellCompartment("neuron",density = 2e5) # 2e5 neurons per meter, 4e10 per sq meter
glia = CellCompartment("glia",density = 2e5) #2e5 glia per meter

neuron_mem = Membrane("neuronal",inside=neuron,outside=ecs,Cm=Cn,phi_m=-70e-3)
glial_mem = Membrane("glial",inside=glia,outside=ecs,Cm=Cg,phi_m=-85e-3)

# Add the compartments to the model
model.addCompartment(ecs,fraction=0.2) # ECS take 20% of the total volume
model.addCompartment(neuron,fraction=0.4) # Neurons take up 40% of the total volume
model.addCompartment(glia,fraction=0.4) # Neurons take up 40% of the total volume

# Add ion species
ecs.addSpecies(K,Ke0,name='K_e')
ecs.addSpecies(Cl,Cle0,name='Cl_e')
ecs.addSpecies(Na,Nae0,name='Na_e')
ecs.addSpecies(Ca,Cae0,name='Ca_e')
ecs.addSpecies(Glu,0.0,name = "g_e")

neuron.addSpecies(K,Ki0,0,'K_n')
neuron.addSpecies(Na,Nai0,0,'Na_n')
neuron.addSpecies(Cl,Cli0,0,'Cl_n')
neuron.addSpecies(Ca,Cai0,0,'Ca_n')
neuron.addSpecies(Glu,1e-6,name = "g_n")


glia.addSpecies(K,Ki0,name='K_g')
glia.addSpecies(Na,Nai0,name='Na_g')
glia.addSpecies(Cl,Cli0,name='Cl_g')
glia.addSpecies(Ca,Cai0,0,'Ca_g')

# add channels
neuron_mem.addChannel(NaTChannel(),10000.) # 100 per neuron?
neuron_mem.addChannel(NaPChannel(),2000.) # 2 per neuron
neuron_mem.addChannel(KDRChannel(),10000.) # number of channels per neuron
neuron_mem.addChannel(CaPChannel(),10000.) # number of channels per neuron
neuron_mem.addChannel(CaLChannel(),10000.) # number of channels per neuron

neuron_mem.addChannel(KIRChannel(),10000.)
#neuron_mem.addChannel(NMDAChannel(),10.)
neuron_mem.addChannel(gNMDAChannel(),10000.)

neuron_mem.addChannel(PMCAPump(),20000) # PMCA pump
neuron_mem.addChannel(NaCaExchangePump(),20000) # sodium-calcium exchanger
neuron_mem.addChannel(NaKATPasePump(),50000.) # 5000 ATPase per neuron
neuron_mem.addChannel(NonSpecificChlorideChannel(-0.07),40000)

glial_mem.addChannel(KIRChannel(),10000) # KIR Channel
glial_mem.addChannel(NaKATPasePump(),100000.) # 10000000 ATPase per glia
glial_mem.addChannel(PMCAPump(),20000)
glial_mem.addChannel(NonSpecificChlorideChannel(-0.085),10000)
glial_mem.addChannel(CaPChannel(),10000.) # number of channels per neuron
glial_mem.addChannel(CaLChannel(),10000.) # number of channels per neuron


model.addMembrane(neuron_mem)
model.addMembrane(glial_mem)

neuron_mem.addLeakChannels()
neuron.balanceWith(ecs)
glial_mem.addLeakChannels()
glia.balanceWith(ecs)

model.assembleSystem()

Ion: Ca2+, P_leak: 3.93E-09
Ion: Na+, P_leak: 1.10E-10
Ion: K+, P_leak: 2.13E-09
Ion: Ca2+, P_leak: 3.25E-09
Ion: K+, P_leak: 1.11E-08
Adding a leak channel does nothing to balance Cl-
Ion: Na+, P_leak: 2.43E-10


In [3]:
y = model.getInternalVars()
#%timeit model.ode_rhs(0,y)

In [4]:
cProfile.run('model.ode_rhs(0,y)')

         661 function calls in 0.013 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.013    0.013 <string>:1(<module>)
       12    0.000    0.000    0.000    0.000 _methods.py:31(_sum)
       13    0.000    0.000    0.000    0.000 channel.py:105(get_h)
       16    0.000    0.000    0.000    0.000 channel.py:121(get_m)
        8    0.001    0.000    0.001    0.000 channel.py:136(permeability)
        8    0.000    0.000    0.000    0.000 channel.py:141(<dictcomp>)
        8    0.000    0.000    0.001    0.000 channel.py:226(mdot)
        5    0.000    0.000    0.001    0.000 channel.py:233(hdot)
        8    0.000    0.000    0.002    0.000 channel.py:315(get_dot_InternalVars)
        6    0.000    0.000    0.000    0.000 channel.py:338(permeability)
       23    0.000    0.000    0.000    0.000 channel.py:38(water_permeability)
        1    0.000    0.000    0.000    0.000 channels.py:100(

In [5]:
cProfile.run('neuron_mem.phi(y)')

         3 function calls in 0.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 membrane.py:248(phi)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [6]:
graphviz = GraphvizOutput()
graphviz.output_file = 'ode_rhs.pdf'
graphviz.output_type = 'pdf'
with PyCallGraph(output=graphviz):
    model.ode_rhs(0,y)

In [7]:
?GraphvizOutput

In [8]:
graphviz2 = GraphvizOutput()
graphviz2.output_file = 'currents.pdf'
graphviz2.output_type = 'pdf'
with PyCallGraph(output=graphviz2):
    neuron_mem.currents(y)

In [9]:
val = ecs.value(K)
val2 = np.zeros(len(val))

In [10]:
%timeit model.ode_rhs(0,y)

100 loops, best of 3: 10.1 ms per loop


In [11]:
ecs.diffusivities[K]

1.96e-09